In [93]:
#Hyperparameter Tuning a Neural Network

In [48]:
 import pandas as pd
 import numpy as np

In [49]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [50]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [51]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [53]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [54]:
x.shape

(768, 8)

In [55]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [56]:
 import tensorflow
 from tensorflow import keras
 from keras import Sequential
 from keras.layers import Dense,Dropout

In [57]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [58]:
model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 11ms/step - loss: 0.7468 - accuracy: 0.4055 - val_loss: 0.6909 - val_accuracy: 0.5519
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6963 - accuracy: 0.5668 - val_loss: 0.6457 - val_accuracy: 0.7273
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6568 - accuracy: 0.6661 - val_loss: 0.6085 - val_accuracy: 0.7468
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6236 - accuracy: 0.7052 - val_loss: 0.5775 - val_accuracy: 0.7857
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5959 - accuracy: 0.7264 - val_loss: 0.5515 - val_accuracy: 0.8052
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5716 - accuracy: 0.7296 - val_loss: 0.5303 - val_accuracy: 0.8052
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5509 - accuracy: 0.7427 - val_loss: 0.5114 - val_accuracy: 0.8052
Epoch 8/100


In [59]:
# 1. How ro select appropriate optimizer

In [60]:
pip install -U keras-tuner

In [61]:
import kerastuner as kt

In [62]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [63]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [64]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [65]:
 tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [66]:
model = tuner.get_best_models(num_models=1)[0]

In [67]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 20ms/step - loss: 0.5360 - accuracy: 0.7427 - val_loss: 0.4963 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5190 - accuracy: 0.7476 - val_loss: 0.4874 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5058 - accuracy: 0.7590 - val_loss: 0.4810 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4964 - accuracy: 0.7606 - val_loss: 0.4775 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4889 - accuracy: 0.7590 - val_loss: 0.4742 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4827 - accuracy: 0.7606 - val_loss: 0.4718 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4777 - accuracy: 0.7606 - val_loss: 0.4709 - val_accuracy: 0.7857
Epoch 14

In [69]:
# 2. Number of nodes in a layer

In [70]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value =8,max_value=128,step =8)

  model.add(Dense(units = units,activation='relu',input_dim =8))
  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [71]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'Rishab')

Reloading Tuner from mydir/Rishab/tuner0.json


In [72]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [73]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [74]:
model = tuner.get_best_models(num_models=1)[0]

In [75]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5321 - accuracy: 0.7557 - val_loss: 0.5163 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5062 - accuracy: 0.7704 - val_loss: 0.4992 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4896 - accuracy: 0.7736 - val_loss: 0.4891 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 12ms/step - loss: 0.4788 - accuracy: 0.7736 - val_loss: 0.4817 - val_accuracy: 0.8182
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4701 - accuracy: 0.7752 - val_loss: 0.4778 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4641 - accuracy: 0.7736 - val_loss: 0.4749 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4600 - accuracy: 0.7769 - val_loss: 0.4732 - val_accuracy: 0.7987
Epoch 1

In [76]:
# 3. How to select number of layers

In [77]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range (hp.Int('num_layers',min_value=1,max_value=5)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [78]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'num_layers')

Reloading Tuner from mydir/num_layers/tuner0.json


In [79]:
tuner.search(x_train,y_train,epochs =5,validation_data=(x_test,y_test))

In [80]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [81]:
model = tuner.get_best_models(num_models=1)[0]

In [82]:
model.fit(x_train,y_train,epochs=100,initial_epoch=6,batch_size=32,validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 4s 40ms/step - loss: 0.4369 - accuracy: 0.7883 - val_loss: 0.5024 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4257 - accuracy: 0.7964 - val_loss: 0.5337 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4229 - accuracy: 0.8013 - val_loss: 0.5266 - val_accuracy: 0.7403
Epoch 10/100
20/20 [==============================] - 0s 13ms/step - loss: 0.4061 - accuracy: 0.8111 - val_loss: 0.5301 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 13ms/step - loss: 0.3982 - accuracy: 0.8094 - val_loss: 0.5207 - val_accuracy: 0.7597
Epoch 12/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3890 - accuracy: 0.8176 - val_loss: 0.5202 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3683 - accuracy: 0.8420 - val_loss: 0.5333 - val_accuracy: 0.7727
Epoc

In [83]:
# 4. All in one model

In [84]:
def build_model(hp):
  model = Sequential()
  counter = 0
  units = hp.Int('units',min_value =8,max_value=128,step =8)

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:
      model.add(
          Dense(
              hp.Int('units'+ str(i),min_value = 8, max_value =128,step = 8),
              activation= hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(
          Dense(
              hp.Int('units'+ str(i),min_value = 8, max_value =128,step = 8),
              activation= hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1

    model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model





In [85]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=3,
                        directory = 'mydir',
                        project_name = 'final1')

Reloading Tuner from mydir/final/tuner0.json


In [86]:
tuner.search(x_train,y_train,epochs =5,validation_data=(x_test,y_test))

In [87]:
tuner.get_best_hyperparameters()[0].values

{'units': 16,
 'num_layers': 8,
 'units0': 104,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 48,
 'activation1': 'sigmoid',
 'units2': 32,
 'activation2': 'tanh',
 'units3': 32,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'relu'}

In [88]:
model = tuner.get_best_models(num_models=1)[0]

In [90]:
model.fit(x_train,y_train,epochs=200,initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/200
20/20 [==============================] - 0s 11ms/step - loss: 0.6474 - accuracy: 0.6531 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 9ms/step - loss: 0.6471 - accuracy: 0.6531 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 9ms/step - loss: 0.6447 - accuracy: 0.6531 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 8ms/step - loss: 0.6428 - accuracy: 0.6531 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 9ms/step - loss: 0.6456 - accuracy: 0.6531 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 9ms/step - loss: 0.6470 - accuracy: 0.6531 - val_loss: 0.6520 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 12ms/step - loss: 0.6439 - accuracy: 0.6531 - val_loss: 0.6521 - val_accuracy: 0.6429
Epoch 1

In [91]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dropout (Dropout)           (None, 104)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 105       
                                                                 
 dense_2 (Dense)             (None, 48)                96        
                                                                 
 dropout_1 (Dropout)         (None, 48)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 49        
                                                                 
 dense_4 (Dense)             (None, 32)                6